In [7]:
import cv2
import mediapipe as mp
import time

In [8]:
cap = cv2.VideoCapture(r"E:\python\segmentation\Computer Vision\Computer Vision\Data\Hand.mp4")
cap.set(3, 720)
cap.set(4, 640)

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

pre_time = 0
curr_time = 0

In [5]:
import cv2
import mediapipe as mp
import time
import math

class Hand_detector():
    def __init__(self, mode=False, max_hands=2, model_complexity=1, detection_conf=0.5, track_conf=0.5):
        self.mode = mode
        self.max_hands = max_hands
        self.model_complexity = model_complexity
        self.detection_conf = detection_conf
        self.track_conf = track_conf
        
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(self.mode, self.max_hands, self.model_complexity,
                                         self.detection_conf, self.track_conf)
        self.mp_draw = mp.solutions.drawing_utils
        self.tipIds = [4, 8, 12, 16, 20]

    def find_hands(self, img, draw=True): 

        self.image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(self.image_rgb)
        #print(results.multi_hand_landmarks)
        if self.results.multi_hand_landmarks:
            for hands_lms in self.results.multi_hand_landmarks:
                if draw:
                        
                    self.mp_draw.draw_landmarks(img, hands_lms,
                                                self.mp_hands.HAND_CONNECTIONS)
        return img

    def find_position(self, img, hand_num=0, draw=True):
        x_list = []
        y_list = []
        bbox = []
        
        self.landmark_list = []
        
        if self.results.multi_hand_landmarks:
            my_hand = self.results.multi_hand_landmarks[hand_num]
            
            for id, lm in enumerate(my_hand.landmark):
                #print(id, landmark)
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                x_list.append(cx)
                y_list.append(cy)
                #print(id, cx, cy)
                self.landmark_list.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (0, 255, 0), cv2.FILLED)

            x_min, x_max = min(x_list), max(x_list)
            y_min, y_max = min(y_list), max(y_list)
            bbox = x_min, y_min, x_max, y_max

            if draw:
                cv2.rectangle(img, (x_min - 20, y_min - 20), (x_max + 20, y_max + 20),
                                 (0, 255, 0), 2)

        return self.landmark_list, bbox

    def finger_up(self):
        fingers = []

        # Thumb
        if self.landmark_list[self.tipIds[0]][1] > self.landmark_list[self.tipIds[0] - 1][1]:
            fingers.append(1)

        else:
            fingers.append(0)

        for id in range(1, 5):
            if self.landmark_list[self.tipIds[id]][2] < self.landmark_list[self.tipIds[id] - 2][2]:
                fingers.append(1)

            else:
                fingers.append(0)

        return fingers

    def find_distance(self, p1, p2, img, draw=True, r=15, thickness=3):
        x1, y1 = self.landmark_list[p1][1:]
        x2, y2 = self.landmark_list[p2][1:]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        if draw:
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), thickness)
            cv2.circle(img, (x1, y1), r, (0, 255, 0), cv2.FIELLD)
            cv2.circle(img, (x2, y2), r, (0, 255, 0), cv2.FIELLD)
            cv2.circle(img, (cx, cy), r, (0, 255, 0), cv2.FIELLD)

        length = math.hypot(x2 - x1, y2 - y1)

        return length, img, [x1, y1, x2, y2, cx, cy]

In [6]:
def main():
    pre_time = 0
    curr_time = 0

    cap = cv2.VideoCapture(r"E:\python\segmentation\Computer Vision\Computer Vision\Data\Hand.mp4")

    detectore = Hand_detector()
    while True:
        success, img = cap.read()
        img = detectore.find_hands(img),#draw=False)
        print(img)
        landmark_list = detectore.find_position(img)#, draw=False)
        if len(landmark_list) != 0:
            print(landmark_list[4])
        
        curr_time = time.time()
        fps = 1 / (curr_time - pre_time)
        pre_time = curr_time
    
        cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN,
                   3, (255, 255, 0), 2)
    
        cv2.imshow("image", img)
        cv2.waitKey(1)

if __name__ == "__main__":
    main()

(array([[[ 78, 117, 102],
        [ 77, 116, 101],
        [ 78, 117, 102],
        ...,
        [ 68, 103,  88],
        [ 68, 103,  88],
        [ 64,  99,  84]],

       [[ 78, 117, 102],
        [ 77, 116, 101],
        [ 78, 117, 102],
        ...,
        [ 67, 102,  87],
        [ 67, 102,  87],
        [ 66, 101,  86]],

       [[ 74, 113,  98],
        [ 74, 113,  98],
        [ 79, 118, 103],
        ...,
        [ 66, 101,  86],
        [ 66, 101,  86],
        [ 67, 102,  87]],

       ...,

       [[102, 144, 128],
        [104, 146, 130],
        [105, 147, 131],
        ...,
        [ 91, 134, 114],
        [ 90, 133, 113],
        [ 86, 129, 109]],

       [[102, 144, 128],
        [105, 147, 131],
        [106, 148, 132],
        ...,
        [ 88, 131, 111],
        [ 86, 129, 109],
        [ 89, 132, 112]],

       [[102, 144, 128],
        [105, 147, 131],
        [107, 149, 133],
        ...,
        [ 83, 126, 106],
        [ 79, 122, 102],
        [ 81, 124, 104]

AttributeError: 'tuple' object has no attribute 'shape'

In [3]:
pre_time = 0
curr_time = 0

cap = cv2.VideoCapture(r"E:\python\segmentation\Computer Vision\Computer Vision\Data\Hand.mp4")

detectore = Hand_detector()
while True:
    success, img = cap.read()
    img = cv2.resize(img, (720, 720))
    img = detectore.find_hands(img)
    landmark_list = detectore.find_position(img)
    if len(landmark_list) != 0:
        print(landmark_list[4])
        
    curr_time = time.time()
    fps = 1 / (curr_time - pre_time)
    pre_time = curr_time
    
    cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN,
                3, (255, 255, 0), 2)
    
    cv2.imshow("image", img)
    cv2.waitKey(1)

ValueError: min() arg is an empty sequence